# Load DFT data into a more manageable format

#### Import packages

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase.db import connect
import sys
import os

In [47]:
# Attach Jacks formic acid data
formic_acid_data_path = "../Experiments/AgAuCuPdPt_data/1_databases/relaxed/"
#sys.path.append()

In [9]:
os.listdir(formic_acid_data_path)

['single_element_H_out.db',
 'HCOOH_O_adsorbed_out.db',
 'H_out.db',
 '.DS_Store',
 'COOH_H_on_O_OO_adsorbed_out.db',
 'single_element_COOH_COH_adsorbed_out.db',
 'COOH_O_adsorbed_out.db',
 'CO_out.db',
 'single_element_COOH_H_CO_adsorbed_out.db',
 'single_element_COOH_C_adsorbed_out.db',
 'single_element_COOH_CO_adsorbed_out.db',
 'single_element_CO_out.db',
 'split_databases.py',
 'single_elements_COOH_CO_adsorbed_out.db',
 'molecules_out.db',
 'single_element_COOH_H_on_O_OO_adsorbed_out.db',
 'single_element_slabs_out.db',
 'single_elements_COOH_COH_adsorbed_out.db',
 'single_element_COOH_H_CO_and_H2O_like_out.db',
 'single_element_HCOOH_O_adsorbed_out.db',
 'slabs_out.db',
 'single_element_COOH_O_adsorbed_out.db',
 'single_element_COOH_H_COH_adsorbed_out.db',
 'COOH_CO_adsorbed_out.db',
 'COOH_H_on_C_OO_adsorbed_out.db',
 'COOH_C_adsorbed_out.db',
 'COOH_COH_adsorbed_out.db']

### Sort through data

In [24]:
single_element_dbs = []
HAE_dbs = []

for filename in os.listdir(formic_acid_data_path):
    if filename[-3:] == ".db":
        if filename[0:14] == "single_element":
            single_element_dbs.append(filename)
        if filename[0] == "H" or filename[0] == "C":
            HAE_dbs.append(filename)

In [25]:
single_element_dbs

['single_element_H_out.db',
 'single_element_COOH_COH_adsorbed_out.db',
 'single_element_COOH_H_CO_adsorbed_out.db',
 'single_element_COOH_C_adsorbed_out.db',
 'single_element_COOH_CO_adsorbed_out.db',
 'single_element_CO_out.db',
 'single_elements_COOH_CO_adsorbed_out.db',
 'single_element_COOH_H_on_O_OO_adsorbed_out.db',
 'single_element_slabs_out.db',
 'single_elements_COOH_COH_adsorbed_out.db',
 'single_element_COOH_H_CO_and_H2O_like_out.db',
 'single_element_HCOOH_O_adsorbed_out.db',
 'single_element_COOH_O_adsorbed_out.db',
 'single_element_COOH_H_COH_adsorbed_out.db']

In [26]:
HAE_dbs

['HCOOH_O_adsorbed_out.db',
 'H_out.db',
 'COOH_H_on_O_OO_adsorbed_out.db',
 'COOH_O_adsorbed_out.db',
 'CO_out.db',
 'COOH_CO_adsorbed_out.db',
 'COOH_H_on_C_OO_adsorbed_out.db',
 'COOH_C_adsorbed_out.db',
 'COOH_COH_adsorbed_out.db']

### Extract energy

In [32]:
def fetch_energies(db_folder, filename):
    Dbong = connect(db_folder + filename)
    energy_list = []
    for row in Dbong.select("energy"):
        energy_list.append(row.energy)
    return energy_list

In [33]:
fetch_energies(formic_acid_data_path, HAE_dbs[0])

[]

In [50]:
print(HAE_dbs[0])
print(HAE_dbs[1])

HCOOH_O_adsorbed_out.db
H_out.db


In [48]:
Dbong = connect(formic_acid_data_path + HAE_dbs[0])
energy_list = []
for row in Dbong.select("energy"):
    print(row)
    energy_list.append(row.energy)

<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag5Au7Cu10Pd12Pt11CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag13Au5Cu12Pd7Pt8CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag13Au5Cu12Pd7Pt8CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag13Au5Cu12Pd7Pt8CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag13Au5Cu12Pd7Pt8CH2O2, keys=idx,slab_idx,site_idx>
<AtomsRow: formula=Ag13Au5Cu12Pd7Pt8CH2O2, keys=idx,slab_idx,site_id

In [51]:
print(len(energy_list))

27


In [42]:
Dbong.keys()

AttributeError: 'SQLite3Database' object has no attribute 'keys'

In [46]:
Dbong = connect(formic_acid_data_path + HAE_dbs[0])
for row in Dbong.get():
    print(row)

KeyError: 'no match'

In [41]:
for row in Dbong.select("energy"):
    print(row)

In [ ]:
H_out = connect(db_folder + "H_out.db")

for row in H_out.select("energy"): #Select the rows that have an energy

    energy_list.append(row.energy)